In [ ]:
import sys
import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM

import index_tracker as track   
import csv
from keras.callbacks import LearningRateScheduler
import cv2
import scipy
import h5py
import skimage
import time

# set seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

from semseg import utils, resnet, detection, losses, data_generator as dg, visualize, metrics
from semseg.config import Config

from semseg import semseg_model as modellib
import dicom_to_numpy as dtn
import matplotlib.pyplot as plt

from semseg import testing_semseg

**1. Set training hyperparameters**

In [ ]:
class TumorConfig(Config):    
    NAME = 'lung_tumor'
    LOSS_WEIGHTS = {           
        "rpn_mask_loss": 1.        
    }
    LEARNING_RATE = 0.001
    IMAGES_PER_GPU = 1
    TRAIN_BN = True
    IMAGE_HEIGHT = 384
    IMAGE_WIDTH = 384
    IMAGE_SHAPE = [384, 384, 1]
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 181   
    
    IMAGE_CHANNEL_COUNT = 1
   

# SET UP CONFIG
config = TumorConfig()

**2. Load DRR test datasets**

In [ ]:
data_path = ''
patient_name = 'patient1'
model = 'semseg'
data_mode = 'standard'
drr_data_path = data_path + patient_name + '/models/'+ model + '/' + data_mode +'/dataset/'
# Directory to save logs and trained model
LOG_DIR =  data_path + patient_name + '/models/' + model + '/' + data_mode +'/logs/'

test_imgs = dtn.load_data(drr_data_path + 'test_imgs.hdf5', 'test_imgs')
test_labs = dtn.load_data(drr_data_path + 'test_labs.hdf5', 'test_labs')

**3. Preprocess loaded datasets**

In [ ]:
for i in range(test_labs.shape[0]):
    for j in range(test_labs.shape[1]):
        test_labs[i, j] = fit_ellipse(test_labs[i, j])        
test_imgs = (test_imgs+1)/2.

# TRAIN MODEL
s = test_imgs.shape
test_imgs = np.reshape(test_imgs, (-1, s[2], s[3]))[:, :, 64:-64]
test_labs = np.reshape(test_labs, (-1, s[2], s[3]))[:, :, 64:-64]

**4. Load trained model**

In [ ]:
model = modellib.SiamMask(mode="inference", config=config,
                                      model_dir=LOG_DIR)


weights_path = model.find_last()
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True) 

**5. Test model on DRRs**

In [ ]:
start = time.time()        
outputs = model.test(test_imgs[0:], labs = test_labs[0:], verbose = 0)
stop = time.time()
print(stop-start)
gt_bboxs = outputs[0]['gt_bboxs']
pred_bboxs = outputs[1]['pred_bboxs'][:, np.newaxis, :]
gt_masks = outputs[0]['gt_masks']
pred_masks = outputs[1]['pred_masks']
pred_centroids = outputs[1]['pred_centroid']

**6. Compute metrics**

In [ ]:
cor_z, cor_x = metrics.concordance_correlation_coefficient(gt_bboxs, pred_centroids)
z_mad, z_std, x_mad, x_std = metrics.mad(gt_bboxs, pred_centroids)    
z_mad, z_std, x_mad, x_std = z_mad/3, z_std/3, x_mad/3, x_std/3 

**7. Load X-Rays**

In [ ]:
data_path = ''
patient_name = 'patient2'
xray_data_path = data_path + patient_name + '/X_rays/'

xrays = load_data(xray_data_path + 'patient2_x_rays_1.h5', 'img')
angles = load_data(xray_data_path + 'patient2_x_rays_1.h5', 'rot')
angles = angles-270

**8. Preprocess X-Rays**

In [ ]:
xrays = utils.log_transform(xrays)

shape = np.array(xrays.shape[1:])
new_shape = shape//2
xrays = xrays[:, shape[0]//2-new_shape[0]//2:shape[0]//2+new_shape[0]//2, 
                shape[1]//2-new_shape[1]//2:shape[1]//2+new_shape[1]//2] 

xrays = (xrays-np.min(xrays))/(np.max(xrays)-np.min(xrays))
xrays = utils.gamma_transform(xrays, 5)

xrays_semseg = xrays[..., 64:-64]

**9. Test model on X-Rays**

In [ ]:
outputs_xrays = model.test(xrays_semseg[0:], labs=None, verbose = 0)

pred_masks = np.round(outputs_xrays[1]['pred_masks']).astype('uint8')
pred_centers = np.round(outputs_xrays[1]['pred_centroid']).astype('int32')
pred_bboxs = np.concatenate((pred_centers-32, pred_centers+32), axis = 1)
pred_bboxs = pred_bboxs[np.newaxis, :, np.newaxis, :]